# https://medium.com/coinmonks/word-level-lstm-text-generator-creating-automatic-song-lyrics-with-neural-networks-b8a1617104fb
### get the pos in order, what occurs
### amd then also get words for those pos and fill in the blanks to make a new one


### train with sequences of words and then guess the target next word

### The idea is that after many epochs the RNN will learn “the style” of how the corpus is written, trying to adjust the weights of the network to predict the next word given a sequence of the N previous words.


In [2]:
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import to_categorical
from  tensorflow.keras.preprocessing import text_dataset_from_directory
import tensorflow
import numpy
from keras.preprocessing.sequence import pad_sequences

# from tensorflow.keras.metrics import 


## Reading the corpus, split into words
### The first step is to read the corpus and split it into words.

## Note the call to .replace(‘\n’, ‘ \n ‘, this is because we want the newline as a word. The idea behind this is that we are also leaving to the network the decision on when to start a new line (after several words). After this, text_in_words is a big array containing all the corpus, word by word.

# https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

# https://medium.com/@enriqueav/update-automatic-song-lyrics-creator-with-word-embeddings-e30de94db8d1

### A trained layer converts the words in the vocabulary in a way that the ones with similar use will be close in their vectorial representation. This means, for instance, that words representing colors: “white”, “red”, “yellow”, etc will be close among them.

### https://github.com/ApurbaSengupta/Lyrics-Generation-using-BERT/blob/master/code/reconstruction.ipynb


one idea is to take all of the sentences from a given genre and then use word2vec to find simmilar senteacnes and swamp them around 
https://datascience.stackexchange.com/questions/23969/sentence-similarity-prediction

another is to switch out word for word, start at first word guess the second then third and so on (replace all words with simmilar words)

another is to remove certain words and then replcae those with a guess based on the previous context

https://datascience.stackexchange.com/questions/9785/predicting-a-word-using-word2vec-model

use word2vec to o thro


## imports

In [3]:
import gensim.models
import os
from gensim.models import Phrases
import logging
import re, string
import nltk
import io
# import gensim.downloader as api
# # googleWordVec = api.load('word2vec-google-news-300')
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# method 1: keep sentance structure (leave stopwords) but compute new words for each using missing word (word2vec) - gram

### import the data

In [4]:
def importData(directory: str) -> list:
    with open(directory) as f:
        return f.read()

def getAllFilesInFolder(dir: str) -> list:
    return os.listdir(dir)

def cleanText(text: str):
    splitText = text.split(" ")
    return splitText


In [5]:
countrySongs = []
for file in getAllFilesInFolder("./data/Country"):
    filePath = "./data/Country/{}".format(file)
    if "DS_Store" not in filePath:
        countrySongs.append(importData(filePath))

len(countrySongs)


100

In [6]:

# source text -> allCountryLyrics
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(countrySongs)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# Transforms each text in texts to a sequence of integers.
encoded = tokenizer.texts_to_sequences(countrySongs)
len(encoded)


Vocabulary Size: 2803


100

In [7]:

# create word -> word sequences 
# get sequences of words together, so given "a b c d"
#  you should give [a,b], [b,c], [c,d] all the adjacent pairs
sequences = []
for i in range(len(encoded)):
	for j in range(3, len(encoded[i])):
		sequence = encoded[i][j-3:j+1]
		sequences.append(sequence)
		
		
print('Total Sequences: %d' % len(sequences))
sequences

Total Sequences: 25267


[[2, 458, 14, 516],
 [458, 14, 516, 2],
 [14, 516, 2, 32],
 [516, 2, 32, 408],
 [2, 32, 408, 3],
 [32, 408, 3, 5],
 [408, 3, 5, 459],
 [3, 5, 459, 460],
 [5, 459, 460, 1439],
 [459, 460, 1439, 34],
 [460, 1439, 34, 1],
 [1439, 34, 1, 161],
 [34, 1, 161, 118],
 [1, 161, 118, 169],
 [161, 118, 169, 18],
 [118, 169, 18, 5],
 [169, 18, 5, 43],
 [18, 5, 43, 320],
 [5, 43, 320, 112],
 [43, 320, 112, 1440],
 [320, 112, 1440, 3],
 [112, 1440, 3, 89],
 [1440, 3, 89, 3],
 [3, 89, 3, 169],
 [89, 3, 169, 94],
 [3, 169, 94, 38],
 [169, 94, 38, 110],
 [94, 38, 110, 4],
 [38, 110, 4, 95],
 [110, 4, 95, 110],
 [4, 95, 110, 156],
 [95, 110, 156, 95],
 [110, 156, 95, 58],
 [156, 95, 58, 2],
 [95, 58, 2, 458],
 [58, 2, 458, 14],
 [2, 458, 14, 516],
 [458, 14, 516, 2],
 [14, 516, 2, 32],
 [516, 2, 32, 408],
 [2, 32, 408, 3],
 [32, 408, 3, 5],
 [408, 3, 5, 459],
 [3, 5, 459, 1104],
 [5, 459, 1104, 73],
 [459, 1104, 73, 261],
 [1104, 73, 261, 3],
 [73, 261, 3, 178],
 [261, 3, 178, 29],
 [3, 178, 29, 188],
 

In [8]:
sequences = numpy.array(sequences)
testData, testLabels = sequences[:,:-1],sequences[:,-1]

In [9]:
sequences

array([[  2, 458,  14, 516],
       [458,  14, 516,   2],
       [ 14, 516,   2,  32],
       ...,
       [  3,  30,   2, 114],
       [ 30,   2, 114,  16],
       [  2, 114,  16,   3]])

In [10]:
testLabels

array([516,   2,  32, ..., 114,  16,   3])

In [11]:
testData

array([[  2, 458,  14],
       [458,  14, 516],
       [ 14, 516,   2],
       ...,
       [  3,  30,   2],
       [ 30,   2, 114],
       [  2, 114,  16]])

In [13]:
import numpy
from keras.preprocessing.sequence import pad_sequences


# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = numpy.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = numpy.zeros((vocab_size, 100))
for word, i in  tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector


Max Sequence Length: 4
Loaded 400001 word vectors.


In [18]:
# create word -> word sequences 
# get sequences of words together, so given "a b c d"
#  you should give [a,b,c], [b,c, d]... all the adjacent triples
sequences = numpy.array(sequences)
testData, testLabels = sequences[:,:-1],sequences[:,-1]
# one hot encode outputs -> Converts a class vector (integers) to binary class matrix.
testLabels= to_categorical(testLabels, num_classes=vocab_size)
# define model
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length-1, trainable=False)
model = Sequential()
model.add(e)
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(testData, testLabels, epochs=50, verbose=2)
# evaluate
# print(generate_seq(model, tokenizer, 'Jack', 6))

2021-10-24 16:23:30.115536: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 100)            280300    
_________________________________________________________________
lstm (LSTM)                  (None, 50)                30200     
_________________________________________________________________
dense (Dense)                (None, 2803)              142953    
Total params: 453,453
Trainable params: 173,153
Non-trainable params: 280,300
_________________________________________________________________
None


2021-10-24 16:23:30.816731: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
790/790 - 4s - loss: 6.3701 - accuracy: 0.0447
Epoch 2/50
790/790 - 3s - loss: 6.0020 - accuracy: 0.0604
Epoch 3/50
790/790 - 3s - loss: 5.8150 - accuracy: 0.0706
Epoch 4/50
790/790 - 3s - loss: 5.6097 - accuracy: 0.0828
Epoch 5/50
790/790 - 3s - loss: 5.4150 - accuracy: 0.0940
Epoch 6/50
790/790 - 3s - loss: 5.2344 - accuracy: 0.1027
Epoch 7/50
790/790 - 2s - loss: 5.0629 - accuracy: 0.1101
Epoch 8/50
790/790 - 2s - loss: 4.8986 - accuracy: 0.1223
Epoch 9/50
790/790 - 2s - loss: 4.7404 - accuracy: 0.1337
Epoch 10/50
790/790 - 2s - loss: 4.5940 - accuracy: 0.1455
Epoch 11/50
790/790 - 2s - loss: 4.4535 - accuracy: 0.1594
Epoch 12/50
790/790 - 2s - loss: 4.3228 - accuracy: 0.1766
Epoch 13/50
790/790 - 2s - loss: 4.1995 - accuracy: 0.1925
Epoch 14/50
790/790 - 3s - loss: 4.0840 - accuracy: 0.2049
Epoch 15/50
790/790 - 3s - loss: 3.9765 - accuracy: 0.2195
Epoch 16/50
790/790 - 2s - loss: 3.8766 - accuracy: 0.2320
Epoch 17/50
790/790 - 3s - loss: 3.7837 - accuracy: 0.2457
Epoch 

In [19]:
result = ""
encodedExample = tokenizer.texts_to_sequences(["the"])[0]
encodedExample = numpy.array(encodedExample)
yhat = numpy.argmax(model.predict(encodedExample), axis=-1)
yhat

out_word = ''
for word, index in tokenizer.word_index.items():
    if index == yhat:
        out_word = word
        break
# append to input
in_text, result = out_word, result + ' ' + out_word
print(in_text)
print(out_word)
print(result)

world
world
 world


In [20]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = numpy.argmax(model.predict(encoded), axis=-1)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [21]:
# print(generate_seq(model, tokenizer, "I", 6))
print(generate_seq(model, tokenizer, max_length-1, 'I am', 70))

#After the model is fit, we test it by passing it a given word from the vocabulary and having the model predict the next word. 
# Here we pass in ‘Jack‘ by encoding it and calling model.predict_classes() to get the integer output for the predicted word. 
# This is then looked up in the vocabulary mapping to give the associated word.

I am but you don't know you do what i love you now i'm probably buying a lot for the world the world oh noi bought to the break ups that didn't break us the break down wrong turn the chance of a lot enough timeand goodbyes for a beautiful gal and a file gumbo 'cause tonight i'm gonna see the angels beckon me from heaven's open door and i can't change


In [36]:
# they went precious times there i know who holds
# who holds my handev'ry step is my wish i
# wish i was born and raisedthe place where i
# where i grew content to be my queen
# my queen you're the one i want to thank

## eval with BLEU score
https://towardsdatascience.com/how-to-evaluate-text-generation-models-metrics-for-automatic-evaluation-of-nlp-models-e1c251b04ec1
